# **Fesmaro 2025**
# **Analisis Big Data: Tahap Modelling**

In [1]:
print("this is just a place holder\nCreate by Research Bocah Gundar")

this is just a place holder
Create by Research Bocah Gundar


# Model and Function Declaration

## Install Library

In [2]:
# Install libraries
!pip install safetensors spacy torchviz wandb --quiet

# Download the required SpaCy model
!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import gdown 
url = "https://drive.google.com/drive/u/4/folders/1La029vITSdOyDC-TaKZ9kxVk0twOneFv"
gdown.download_folder(url)

Retrieving folder contents


Processing file 14XpIdfj6FN9ckvBVrRp9lUa-OtuDIu5O df_processed.csv
Processing file 1ZwRcWT9RubqpZ_qFukgq9ZyO3Ob0I0Q9 df_test.csv
Processing file 1Btr1jmjO2fx8Uh6_4-It79Mf5pLdXQcy df_train.csv
Processing file 1giX5L2NXjxDXepK3wsaRNJjUcSzZf_9P df_val.csv
Processing file 1YioeFhGr0yUFVX3dFcIwzFsPFfd11LKj final_df.csv


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=14XpIdfj6FN9ckvBVrRp9lUa-OtuDIu5O
From (redirected): https://drive.google.com/uc?id=14XpIdfj6FN9ckvBVrRp9lUa-OtuDIu5O&confirm=t&uuid=ec8cb96a-2798-4404-b6a1-85760b87241e
To: /kaggle/working/data/df_processed.csv
100%|██████████| 128M/128M [00:00<00:00, 227MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1ZwRcWT9RubqpZ_qFukgq9ZyO3Ob0I0Q9
To: /kaggle/working/data/df_test.csv
100%|██████████| 12.1M/12.1M [00:00<00:00, 94.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Btr1jmjO2fx8Uh6_4-It79Mf5pLdXQcy
To: /kaggle/working/data/df_train.csv
100%|██████████| 40.8M/40.8M [00:00<00:00, 67.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1giX5L2NXjxDXepK3wsaRNJjUcSzZf_9P
To: /kaggle/working/data/df_val.csv
100%|██████████| 8.06M/8.06M [00:00<00:00, 42.5MB/s]
Downloading...
From: https://drive.google

['/kaggle/working/data/df_processed.csv',
 '/kaggle/working/data/df_test.csv',
 '/kaggle/working/data/df_train.csv',
 '/kaggle/working/data/df_val.csv',
 '/kaggle/working/data/final_df.csv']

## Import Library

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, AutoConfig
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import os
import pickle
import hashlib
import json
from tqdm.notebook import tqdm  # Use notebook-friendly tqdm
from safetensors.torch import save_file, load_file
import wandb
import re
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

# Untuk visualisasi model (opsional, install jika diperlukan)
try:
    import torchviz
except ImportError:
    print("torchviz not found. Install it via 'pip install torchviz' for model visualization.")
    torchviz = None

## Konfigurasi Eksperimen

**PLEASE READ ME!**

**SILAHKAN PILIH SALAH SATU CONFIGURASI MODEL**

In [4]:
# --- 1. Konfigurasi Dual GPU ---
num_gpus = torch.cuda.device_count()
print(f"Found {num_gpus} GPUs.")

Found 2 GPUs.


In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def simple_tokenizer(text):
    return tokenizer.tokenize(text)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

### CNN

In [8]:
config = {
    # Pilihan Model: 'bert', 'lstm', 'cnn', 'gcn', 'bert_lstm_gcn', 'bert_lstm_cnn'
    "model_type": "cnn",
    "bert_model_name": "bert-base-uncased",
    "max_length": 128,
    "embedding_dim": 300, # Untuk model non-BERT (LSTM, CNN, GCN)
    "vocab_size": 10000,  # Perkiraan ukuran vocab untuk model non-BERT
    "hidden_dim": 128,
    "lstm_layers": 1,
    "gcn_layers": 2,
    "cnn_filters": 256,
    "cnn_kernel_sizes": [3, 4, 5],
    "dropout": 0.3,
    "num_classes": 2, # Akan diupdate setelah LabelEncoder
    "epochs": 3, # Kurangi untuk testing cepat
    # ====> IMPORTANT: Increase batch size for multi-GPU training <====
    # Original: 16. Try doubling it for 2 GPUs, adjust based on memoxry.
    "batch_size": 128, # Total batch size across all GPUs
    "learning_rate": 2e-5, # LR umum untuk fine-tuning BERT
    "non_bert_lr": 1e-3, # LR umum untuk model non-BERT
    "cache_dir": "./feature_cache",
    "log_file": "training_log.json",
    "model_save_path": "best_model.safetensors",
    # Use 'cuda' which defaults to cuda:0 (primary GPU for DataParallel)
    "device": 'cuda' if torch.cuda.is_available() else 'cpu',
    "use_linguistic_features": True, # Set False jika tidak ingin menggunakan fitur linguistik
    "linguistic_feat_dim": 9,
    "freeze_bert": False # Set True untuk freeze bobot BERT (lebih cepat, mungkin akurasi turun)
}

### LSTM

In [9]:
config = {
    # Pilihan Model: 'bert', 'lstm', 'cnn', 'gcn', 'bert_lstm_gcn', 'bert_lstm_cnn'
    "model_type": "lstm",
    "bert_model_name": "bert-base-uncased",
    "max_length": 128,
    "embedding_dim": 300, # Untuk model non-BERT (LSTM, CNN, GCN)
    "vocab_size": 10000,  # Perkiraan ukuran vocab untuk model non-BERT
    "hidden_dim": 128,
    "lstm_layers": 1,
    "gcn_layers": 2,
    "cnn_filters": 256,
    "cnn_kernel_sizes": [3, 4, 5],
    "dropout": 0.3,
    "num_classes": 2, # Akan diupdate setelah LabelEncoder
    "epochs": 3, # Kurangi untuk testing cepat
    # ====> IMPORTANT: Increase batch size for multi-GPU training <====
    # Original: 16. Try doubling it for 2 GPUs, adjust based on memoxry.
    "batch_size": 128, # Total batch size across all GPUs
    "learning_rate": 2e-5, # LR umum untuk fine-tuning BERT
    "non_bert_lr": 1e-3, # LR umum untuk model non-BERT
    "cache_dir": "./feature_cache",
    "log_file": "training_log.json",
    "model_save_path": "best_model.safetensors",
    # Use 'cuda' which defaults to cuda:0 (primary GPU for DataParallel)
    "device": 'cuda' if torch.cuda.is_available() else 'cpu',
    "use_linguistic_features": True, # Set False jika tidak ingin menggunakan fitur linguistik
    "linguistic_feat_dim": 9,
    "freeze_bert": False # Set True untuk freeze bobot BERT (lebih cepat, mungkin akurasi turun)
}

### GCN

In [7]:
config = {
    # Pilihan Model: 'bert', 'lstm', 'cnn', 'gcn', 'bert_lstm_gcn', 'bert_lstm_cnn'
    "model_type": "gcn",
    "bert_model_name": "bert-base-uncased",
    "max_length": 128,
    "embedding_dim": 300, # Untuk model non-BERT (LSTM, CNN, GCN)
    "vocab_size": 10000,  # Perkiraan ukuran vocab untuk model non-BERT
    "hidden_dim": 128,
    "lstm_layers": 1,
    "gcn_layers": 2,
    "cnn_filters": 256,
    "cnn_kernel_sizes": [3, 4, 5],
    "dropout": 0.3,
    "num_classes": 2, # Akan diupdate setelah LabelEncoder
    "epochs": 3, # Kurangi untuk testing cepat
    # ====> IMPORTANT: Increase batch size for multi-GPU training <====
    # Original: 16. Try doubling it for 2 GPUs, adjust based on memoxry.
    "batch_size": 128, # Total batch size across all GPUs
    "learning_rate": 2e-5, # LR umum untuk fine-tuning BERT
    "non_bert_lr": 1e-3, # LR umum untuk model non-BERT
    "cache_dir": "./feature_cache",
    "log_file": "training_log.json",
    "model_save_path": "best_model.safetensors",
    # Use 'cuda' which defaults to cuda:0 (primary GPU for DataParallel)
    "device": 'cuda' if torch.cuda.is_available() else 'cpu',
    "use_linguistic_features": True, # Set False jika tidak ingin menggunakan fitur linguistik
    "linguistic_feat_dim": 9,
    "freeze_bert": False # Set True untuk freeze bobot BERT (lebih cepat, mungkin akurasi turun)
}

### BERT

In [8]:
config = {
    # Pilihan Model: 'bert', 'lstm', 'cnn', 'gcn', 'bert_lstm_gcn', 'bert_lstm_cnn'
    "model_type": "bert",
    "bert_model_name": "bert-base-uncased",
    "max_length": 128,
    "embedding_dim": 300, # Untuk model non-BERT (LSTM, CNN, GCN)
    "vocab_size": 10000,  # Perkiraan ukuran vocab untuk model non-BERT
    "hidden_dim": 128,
    "lstm_layers": 1,
    "gcn_layers": 2,
    "cnn_filters": 256,
    "cnn_kernel_sizes": [3, 4, 5],
    "dropout": 0.3,
    "num_classes": 2, # Akan diupdate setelah LabelEncoder
    "epochs": 3, # Kurangi untuk testing cepat
    # ====> IMPORTANT: Increase batch size for multi-GPU training <====
    # Original: 16. Try doubling it for 2 GPUs, adjust based on memoxry.
    "batch_size": 128, # Total batch size across all GPUs
    "learning_rate": 2e-5, # LR umum untuk fine-tuning BERT
    "non_bert_lr": 1e-3, # LR umum untuk model non-BERT
    "cache_dir": "./feature_cache",
    "log_file": "training_log.json",
    "model_save_path": "best_model.safetensors",
    # Use 'cuda' which defaults to cuda:0 (primary GPU for DataParallel)
    "device": 'cuda' if torch.cuda.is_available() else 'cpu',
    "use_linguistic_features": True, # Set False jika tidak ingin menggunakan fitur linguistik
    "linguistic_feat_dim": 9,
    "freeze_bert": False # Set True untuk freeze bobot BERT (lebih cepat, mungkin akurasi turun)
}

### BERT-LSTM-CNN

In [6]:
config = {
    # Pilihan Model: 'bert', 'lstm', 'cnn', 'gcn', 'bert_lstm_gcn', 'bert_lstm_cnn'
    "model_type": "bert_lstm_cnn",
    "bert_model_name": "bert-base-uncased",
    "max_length": 128,
    "embedding_dim": 300, # Untuk model non-BERT (LSTM, CNN, GCN)
    "vocab_size": 10000,  # Perkiraan ukuran vocab untuk model non-BERT
    "hidden_dim": 128,
    "lstm_layers": 1,
    "gcn_layers": 2,
    "cnn_filters": 256,
    "cnn_kernel_sizes": [3, 4, 5],
    "dropout": 0.3,
    "num_classes": 2, # Akan diupdate setelah LabelEncoder
    "epochs": 3, # Kurangi untuk testing cepat
    # ====> IMPORTANT: Increase batch size for multi-GPU training <====
    # Original: 16. Try doubling it for 2 GPUs, adjust based on memoxry.
    "batch_size": 128, # Total batch size across all GPUs
    "learning_rate": 2e-5, # LR umum untuk fine-tuning BERT
    "non_bert_lr": 1e-3, # LR umum untuk model non-BERT
    "cache_dir": "./feature_cache",
    "log_file": "training_log.json",
    "model_save_path": "best_model.safetensors",
    # Use 'cuda' which defaults to cuda:0 (primary GPU for DataParallel)
    "device": 'cuda' if torch.cuda.is_available() else 'cpu',
    "use_linguistic_features": True, # Set False jika tidak ingin menggunakan fitur linguistik
    "linguistic_feat_dim": 9,
    "freeze_bert": False # Set True untuk freeze bobot BERT (lebih cepat, mungkin akurasi turun)
}

### BERT-LSTM-GCN

In [6]:
config = {
    # Pilihan Model: 'bert', 'lstm', 'cnn', 'gcn', 'bert_lstm_gcn', 'bert_lstm_cnn'
    "model_type": "bert_lstm_gcn",
    "bert_model_name": "bert-base-uncased",
    "max_length": 128,
    "embedding_dim": 300, # Untuk model non-BERT (LSTM, CNN, GCN)
    "vocab_size": 10000,  # Perkiraan ukuran vocab untuk model non-BERT
    "hidden_dim": 128,
    "lstm_layers": 1,
    "gcn_layers": 2,
    "cnn_filters": 256,
    "cnn_kernel_sizes": [3, 4, 5],
    "dropout": 0.3,
    "num_classes": 2, # Akan diupdate setelah LabelEncoder
    "epochs": 3, # Kurangi untuk testing cepat
    # ====> IMPORTANT: Increase batch size for multi-GPU training <====
    # Original: 16. Try doubling it for 2 GPUs, adjust based on memoxry.
    "batch_size": 128, # Total batch size across all GPUs
    "learning_rate": 2e-5, # LR umum untuk fine-tuning BERT
    "non_bert_lr": 1e-3, # LR umum untuk model non-BERT
    "cache_dir": "./feature_cache",
    "log_file": "training_log.json",
    "model_save_path": "best_model.safetensors",
    # Use 'cuda' which defaults to cuda:0 (primary GPU for DataParallel)
    "device": 'cuda' if torch.cuda.is_available() else 'cpu',
    "use_linguistic_features": True, # Set False jika tidak ingin menggunakan fitur linguistik
    "linguistic_feat_dim": 9,
    "freeze_bert": False # Set True untuk freeze bobot BERT (lebih cepat, mungkin akurasi turun)
}

## Data Loading

In [7]:
# Check device and cache
print(f"Using primary device: {config['device']}")
if config['device'] == 'cuda' and num_gpus > 1:
    print(f"Activating DataParallel for {num_gpus} GPUs.")
    # Ensure batch size is divisible by num_gpus for even splits, though DataParallel handles uneven splits too.
    if config['batch_size'] % num_gpus != 0:
        print(f"Warning: Batch size {config['batch_size']} is not perfectly divisible by {num_gpus} GPUs.")


if not os.path.exists(config['cache_dir']):
    os.makedirs(config['cache_dir'])

Using primary device: cuda
Activating DataParallel for 2 GPUs.


In [8]:
# --- 2. Data Loading ---
print("Loading data...")
try:
    final_df = pd.read_csv('./data/final_df.csv')
    df_val = pd.read_csv('./data/df_val.csv')
    df_test = pd.read_csv('./data/df_test.csv')

    # Rename & Select Columns (asumsi kolom sudah benar setelah EDA)
    final_df.rename(columns={'lemmatized_text': 'text'}, inplace=True)
    df_val.rename(columns={'lemmatized_text': 'text'}, inplace=True)
    df_test.rename(columns={'lemmatized_text': 'text'}, inplace=True)

    # Mengubah value label menjadi 'negative' dan 'positive' sebelum encode label
    final_df['label'] = np.where(final_df['label'] == 0, 'negative', 'positive')
    df_val['label'] = np.where(df_val['label'] == 0, 'negative', 'positive')
    df_test['label'] = np.where(df_test['label'] == 0, 'negative', 'positive')

    # Encode Labels (Penting!)
    label_encoder = LabelEncoder()
    final_df['label_encoded'] = label_encoder.fit_transform(final_df['label'])
    df_val['label_encoded'] = label_encoder.transform(df_val['label'])
    df_test['label_encoded'] = label_encoder.transform(df_test['label'])
    config['num_classes'] = len(label_encoder.classes_) # Update num_classes
    print(f"Classes: {label_encoder.classes_} -> {label_encoder.transform(label_encoder.classes_)}")
    print(f"Number of classes: {config['num_classes']}")

except FileNotFoundError as e:
    print(f"Error loading data: {e}. Make sure CSV files are in the './data/' directory.")
    exit()
except KeyError as e:
    print(f"Error: Column {e} not found. Check CSV column names.")
    exit()

Loading data...
Classes: ['negative' 'positive'] -> [0 1]
Number of classes: 2


## Feature Extraction & Preprocessing

In [9]:
# --- 3. Feature Extraction & Preprocessing ---

# Load SpaCy model (hanya jika diperlukan GCN atau linguistic features)
if config['model_type'] in ['gcn', 'bert_lstm_gcn'] or config['use_linguistic_features']:
    try:
        nlp = spacy.load("en_core_web_sm")
    except OSError:
        print("Downloading spacy en_core_web_sm model...")
        spacy.cli.download("en_core_web_sm")
        nlp = spacy.load("en_core_web_sm")
else:
    nlp = None

# Inisialisasi Tokenizer BERT
tokenizer = BertTokenizer.from_pretrained(config['bert_model_name'])

def simple_tokenizer(text):
    """Tokenizer sederhana: lowercase dan split berdasarkan non-alphanumeric."""
    text = text.lower()
    # Simpan tanda baca dasar sebagai token terpisah, split yang lain
    text = re.sub(r"([.!?,'()])", r" \1 ", text)
    tokens = text.split()
    return tokens

def build_vocab(texts, max_vocab_size, min_freq=2):
    """Membangun vocabulary dari list teks."""
    print("Building vocabulary...")
    word_counts = Counter()
    for text in tqdm(texts, desc="Counting words"):
        tokens = simple_tokenizer(str(text)) # Pastikan string
        word_counts.update(tokens)

    # Filter berdasarkan frekuensi minimum
    filtered_counts = {word: count for word, count in word_counts.items() if count >= min_freq}

    # Urutkan berdasarkan frekuensi (descending)
    sorted_words = sorted(filtered_counts.keys(), key=lambda w: word_counts[w], reverse=True)

    # Buat vocab: <PAD>=0, <UNK>=1, lalu kata-kata lainnya
    vocab = {'<PAD>': 0, '<UNK>': 1}
    idx = 2
    for word in sorted_words:
        if idx >= max_vocab_size:
            break
        if word not in vocab: # Seharusnya tidak terjadi dengan sort unik, tapi aman
            vocab[word] = idx
            idx += 1

    actual_vocab_size = len(vocab)
    print(f"Vocabulary built: Size = {actual_vocab_size} (Requested max: {max_vocab_size}, Min freq: {min_freq})")
    print(f"Example vocab items: {list(vocab.items())[:10]} ... {list(vocab.items())[-5:]}")
    return vocab, actual_vocab_size

def pad_sequence(ids, max_len, pad_id):
    """Melakukan padding atau truncation pada sequence ID."""
    if len(ids) < max_len:
        return ids + [pad_id] * (max_len - len(ids))
    else:
        return ids[:max_len]
    
# final_df, df_val, df_test sudah di-load dan di-preprocess labelnya
# --- Tambahan: Bangun Vocabulary (HANYA jika model non-BERT mungkin digunakan) ---
non_bert_models = ['lstm', 'cnn', 'gcn']
vocab = None
pad_id = 0 # Biasanya 0 untuk nn.Embedding padding_idx
unk_id = 1 # Biasanya 1
if any(m == config['model_type'] for m in non_bert_models):
    # Gabungkan semua teks untuk membangun vocab (atau cukup training set)
    all_train_texts = final_df['text'].tolist()
    # Anda bisa juga menambahkan teks validasi/test jika ingin vocab lebih lengkap,
    # tapi umumnya cukup dari training set.
    # all_texts = final_df['text'].tolist() + df_val['text'].tolist() + df_test['text'].tolist()
    # Bangun vocab dari data training
    vocab, actual_vocab_size = build_vocab(
        all_train_texts,
        config['vocab_size'], # Gunakan config['vocab_size'] sebagai batas MAKSIMUM
        min_freq=2 # Atur frekuensi minimum sesuai kebutuhan
    )
    # Update config dengan ukuran vocab aktual yang dihasilkan
    config['vocab_size'] = actual_vocab_size
    print(f"Updated config['vocab_size'] to actual size: {config['vocab_size']}")
else:
    print("Skipping vocabulary building as a non-BERT model is not selected.")
    # Set vocab_size ke nilai default BERT jika hanya BERT yg dipakai (meski tidak relevan)
    # config['vocab_size'] = tokenizer.vocab_size # Atau biarkan saja dari config awal

class FeatureExtractor:
    def __init__(self, config, tokenizer, nlp=None):
        self.config = config
        self.tokenizer = tokenizer
        self.nlp = nlp
        self.cache_dir = config['cache_dir']

    def _get_cache_path(self, text, prefix):
        # Limit text length for hashing to avoid issues with very long texts
        text_snippet = text[:2000]
        text_hash = hashlib.md5(text_snippet.encode()).hexdigest()
        return os.path.join(self.cache_dir, f"{prefix}_{text_hash}.pkl")

    def get_bert_inputs(self, text):
        return self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.config['max_length'],
            return_tensors='pt'
        )

    def get_dependency_graph(self, text, use_cache=True):
        if not self.nlp:
            adj = torch.eye(self.config['max_length'])
            return adj

        # Ensure text is not excessively long for caching key generation
        cache_key_text = str(text)[:2000] # Use a snippet for the hash key
        cache_file = self._get_cache_path(cache_key_text, "dep_graph")

        if use_cache and os.path.exists(cache_file):
            try:
                with open(cache_file, 'rb') as f:
                    return pickle.load(f)
            except Exception as e:
                print(f"Warning: Could not load cache file {cache_file}. Recomputing. Error: {e}")

        # Limit text length for SpaCy processing
        doc = self.nlp(str(text)[:1500])
        max_len = self.config['max_length']
        adj_matrix = np.zeros((max_len, max_len))

        token_map = {}
        spacy_tokens = [token for token in doc if not token.is_space][:max_len]

        valid_indices = set(range(len(spacy_tokens)))

        for i, token in enumerate(spacy_tokens):
            # Use indices relative to the spacy_tokens list
            idx = i
            head_idx_orig = token.head.i # Original index in doc

            # Find the corresponding index in spacy_tokens for the head
            head_idx = -1
            for j, head_token_candidate in enumerate(spacy_tokens):
                if head_token_candidate.i == head_idx_orig:
                    head_idx = j
                    break

            # Add edge if both token and its head are within the first max_len spacy tokens
            if head_idx != -1 and idx < max_len and head_idx < max_len:
                 # Ensure we don't go out of bounds for the adj_matrix
                 adj_matrix[idx, head_idx] = 1
                 adj_matrix[head_idx, idx] = 1 # Undirected graph

        # Add self-loops for tokens present
        num_actual_tokens = len(spacy_tokens)
        adj_matrix[:num_actual_tokens, :num_actual_tokens] += np.eye(num_actual_tokens)

        # Add self-loops for padding positions
        for i in range(num_actual_tokens, max_len):
            adj_matrix[i, i] = 1

        # Normalize (Symmetric)
        rowsum = np.array(adj_matrix.sum(1)).flatten()
        d_inv_sqrt = np.power(rowsum, -0.5)
        d_inv_sqrt[np.isinf(d_inv_sqrt) | np.isnan(d_inv_sqrt)] = 0.
        d_mat_inv_sqrt = np.diag(d_inv_sqrt)
        normalized_adj = d_mat_inv_sqrt.dot(adj_matrix).dot(d_mat_inv_sqrt)

        result = torch.FloatTensor(normalized_adj)

        if use_cache:
            try:
                with open(cache_file, 'wb') as f:
                    pickle.dump(result, f)
            except Exception as e:
                print(f"Warning: Could not save cache file {cache_file}. Error: {e}")

        return result


    def get_linguistic_features(self, text):
        if not self.nlp or not self.config['use_linguistic_features']:
            return torch.zeros(self.config['linguistic_feat_dim'])

        features = {}
        text_limited = str(text)[:1000]
        words = text_limited.split()
        word_count = len(words)

        # Normalize features (mostly between 0 and 1)
        features['text_length'] = min(len(text_limited), 1000) / 1000.0
        features['word_count'] = min(word_count, 200) / 200.0
        features['avg_word_length'] = min(np.mean([len(word) for word in words]) if words else 0, 20) / 20.0
        features['exclamation_count'] = min(text_limited.count('!'), 10) / 10.0
        features['question_count'] = min(text_limited.count('?'), 10) / 10.0
        uppercase_word_count = sum(1 for word in words if word.isupper() and len(word) > 1)
        features['uppercase_word_ratio'] = uppercase_word_count / (word_count + 1e-6) # Ratio is already scaled
        features['uppercase_word_count_norm'] = min(uppercase_word_count, 20) / 20.0 # Absolute count normalized

        positive_words = {'good', 'great', 'excellent', 'amazing', 'wonderful', 'best', 'love',
                        'perfect', 'recommend', 'happy', 'awesome', 'like', 'highly'}
        negative_words = {'bad', 'poor', 'terrible', 'horrible', 'worst', 'waste', 'disappointed',
                        'disappointing', 'difficult', 'hate', 'problem', 'issue', 'fail', 'never', 'not'}

        lower_words = text_limited.lower().split()
        pos_count = sum(1 for word in lower_words if word in positive_words)
        neg_count = sum(1 for word in lower_words if word in negative_words)

        features['positive_word_count_norm'] = min(pos_count, 20) / 20.0
        features['negative_word_count_norm'] = min(neg_count, 20) / 20.0
        # Scaled sentiment score (-1 to 1 approx)
        features['sentiment_score_scaled'] = (pos_count - neg_count) / (word_count + 1e-6)


        # Ensure the correct number of features are returned matching linguistic_feat_dim
        feature_values = [
            features.get('text_length', 0),
            features.get('word_count', 0),
            features.get('avg_word_length', 0),
            features.get('exclamation_count', 0),
            features.get('question_count', 0),
            features.get('uppercase_word_ratio', 0),
            features.get('positive_word_count_norm', 0),
            features.get('negative_word_count_norm', 0),
            features.get('sentiment_score_scaled', 0),
            # Add more features or padding zeros if needed up to linguistic_feat_dim
        ]

        # Pad or truncate to match linguistic_feat_dim
        final_features = feature_values[:self.config['linguistic_feat_dim']]
        if len(final_features) < self.config['linguistic_feat_dim']:
            final_features.extend([0.0] * (self.config['linguistic_feat_dim'] - len(final_features)))

        return torch.FloatTensor(final_features)


feature_extractor = FeatureExtractor(config, tokenizer, nlp)

Skipping vocabulary building as a non-BERT model is not selected.


## Dataset & DataLoader

In [10]:
# --- 4. Dataset & DataLoader ---
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, feature_extractor, config, vocab=None, pad_id=0, unk_id=1): # Tambahkan vocab, pad_id, unk_id
        self.texts = texts
        self.labels = labels
        self.feature_extractor = feature_extractor
        self.config = config
        self.model_type = config['model_type']
        # Simpan vocab dan ID khusus jika model non-BERT
        self.vocab = vocab
        self.pad_id = pad_id
        self.unk_id = unk_id

        # Validasi: vocab diperlukan untuk model non-BERT
        if self.model_type in ['lstm', 'cnn', 'gcn'] and self.vocab is None:
            raise ValueError(f"Model type '{self.model_type}' requires a vocabulary, but 'vocab' was not provided to SentimentDataset.")


    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx]) # Pastikan string
        label = self.labels[idx]

        item = {'text': text, 'label': torch.tensor(label, dtype=torch.long)}

        # --- Fitur BERT (jika diperlukan) ---
        if 'bert' in self.model_type:
            bert_inputs = self.feature_extractor.get_bert_inputs(text)
            item['input_ids'] = bert_inputs['input_ids'].squeeze(0)
            item['attention_mask'] = bert_inputs['attention_mask'].squeeze(0)

        # --- Fitur GCN (selalu diekstrak jika GCN ada di model type, menggunakan SpaCy) ---
        # Catatan: Untuk GCN murni, adj_matrix ini mungkin tidak sempurna selaras
        # dengan token_ids dari simple_tokenizer. Ini adalah kompromi.
        if 'gcn' in self.model_type:
            item['adj_matrix'] = self.feature_extractor.get_dependency_graph(text)

        # --- Fitur Linguistik (jika diperlukan) ---
        if self.config['use_linguistic_features'] and self.model_type in ['bert_lstm_gcn', 'bert_lstm_cnn']:
            item['linguistic_features'] = self.feature_extractor.get_linguistic_features(text)

        # --- Fitur untuk model non-BERT (LSTM, CNN, GCN murni) ---
        if self.model_type in ['lstm', 'cnn', 'gcn']:
            # 1. Tokenisasi Non-BERT
            tokens = simple_tokenizer(text)

            # 2. Konversi Token ke ID (menggunakan vocab yang diberikan)
            token_ids = [self.vocab.get(token, self.unk_id) for token in tokens] # Gunakan unk_id jika OOV

            # 3. Padding / Truncation
            max_len = self.config['max_length']
            padded_ids = pad_sequence(token_ids, max_len, self.pad_id)

            # 4. Buat Tensor dan tambahkan ke item
            item['token_ids'] = torch.tensor(padded_ids, dtype=torch.long)

            # (Tidak perlu lagi placeholder atau warning)
            # (adj_matrix sudah ditangani di blok 'gcn' di atas)
        return item


# Buat datasets
print("Creating datasets...")
train_dataset = SentimentDataset(
    final_df['text'].tolist(),
    final_df['label_encoded'].tolist(),
    feature_extractor,
    config,
    vocab=vocab, # Kirim vocab
    pad_id=pad_id, # Kirim pad_id
    unk_id=unk_id  # Kirim unk_id
)
val_dataset = SentimentDataset(
    df_val['text'].tolist(),
    df_val['label_encoded'].tolist(),
    feature_extractor,
    config,
    vocab=vocab, # Kirim vocab
    pad_id=pad_id, # Kirim pad_id
    unk_id=unk_id  # Kirim unk_id
)
test_dataset = SentimentDataset(
    df_test['text'].tolist(),
    df_test['label_encoded'].tolist(),
    feature_extractor,
    config,
    vocab=vocab, # Kirim vocab
    pad_id=pad_id, # Kirim pad_id
    unk_id=unk_id  # Kirim unk_id
)

# Buat dataloaders
# num_workers > 0 can cause issues on Kaggle notebooks, keep it 0 unless you are sure.
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=0)

print("Dataloaders created.")
print(f"Train batches: {len(train_dataloader)}, Val batches: {len(val_dataloader)}, Test batches: {len(test_dataloader)}")


Creating datasets...
Dataloaders created.
Train batches: 782, Val batches: 157, Test batches: 235


## Model Definitions

In [11]:
# --- 5. Model Definitions ---

# 5.1 GCN Layer
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.weight)
        if self.bias is not None:
            torch.nn.init.zeros_(self.bias)

    def forward(self, input, adj):
        # input: [batch_size, seq_len, in_features]
        # adj: [batch_size, seq_len, seq_len]
        support = torch.matmul(input, self.weight) # [batch_size, seq_len, out_features] - matmul handles batch dimension correctly
        output = torch.bmm(adj, support) # [batch_size, seq_len, out_features]

        if self.bias is not None:
             output = output + self.bias # Bias broadcasts correctly over batch and seq_len

        return output

# 5.2 BERT Model
class BERTClassifier(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.bert_model_name = config['bert_model_name']
        self.bert_config = AutoConfig.from_pretrained(self.bert_model_name)
        self.bert = BertModel.from_pretrained(self.bert_model_name, config=self.bert_config)
        self.dropout = nn.Dropout(config['dropout'])
        self.classifier = nn.Linear(self.bert_config.hidden_size, config['num_classes'])

        if config['freeze_bert']:
            print("Freezing BERT parameters.")
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask, **kwargs):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# 5.3 LSTM Model (Standalone - Requires non-BERT input processing)
class LSTMClassifier(nn.Module):
    # ... (Keep definition as is, ensure Dataset provides 'token_ids' if used)
    def __init__(self, config):
            super().__init__()
            self.embedding = nn.Embedding(config['vocab_size'], config['embedding_dim'], padding_idx=0) # Asumsi padding_idx=0
            self.lstm = nn.LSTM(config['embedding_dim'], config['hidden_dim'],
                                num_layers=config['lstm_layers'], bidirectional=True,
                                batch_first=True, dropout=config['dropout'] if config['lstm_layers'] > 1 else 0)
            self.dropout = nn.Dropout(config['dropout'])
            self.classifier = nn.Linear(config['hidden_dim'] * 2, config['num_classes']) # *2 for bidirectional

    def forward(self, token_ids, **kwargs): # Terima token_ids dari Dataset
            # This expects 'token_ids' key in the batch from dataloader
            embedded = self.dropout(self.embedding(token_ids)) # [batch, seq_len, embed_dim]
            lstm_out, (hidden, cell) = self.lstm(embedded) # lstm_out: [batch, seq_len, hidden*2]
            # Concat final forward and backward hidden states
            hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1) # [batch, hidden*2]
            hidden = self.dropout(hidden)
            logits = self.classifier(hidden)
            return logits

# 5.4 CNN Model (Standalone - Requires non-BERT input processing)
class CNNClassifier(nn.Module):
    # ... (Keep definition as is, ensure Dataset provides 'token_ids' if used)
    def __init__(self, config):
        super().__init__()
        self.embedding = nn.Embedding(config['vocab_size'], config['embedding_dim'], padding_idx=0)
        # Conv1d expects (batch, embed_dim, seq_len)
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=config['embedding_dim'],
                      out_channels=config['cnn_filters'],
                      kernel_size=k)
            for k in config['cnn_kernel_sizes']
        ])
        self.dropout = nn.Dropout(config['dropout'])
        self.classifier = nn.Linear(len(config['cnn_kernel_sizes']) * config['cnn_filters'], config['num_classes'])

    def forward(self, token_ids, **kwargs):
        # This expects 'token_ids' key in the batch from dataloader
        embedded = self.embedding(token_ids) # [batch, seq_len, embed_dim]
        embedded = embedded.permute(0, 2, 1) # [batch, embed_dim, seq_len]
        embedded = self.dropout(embedded)

        conved = [F.leaky_relu(conv(embedded)) for conv in self.convs] # list of [batch, filters, seq_len-k+1]
        # Max-over-time pooling
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved] # list of [batch, filters]
        cat = torch.cat(pooled, dim=1) # [batch, filters * num_kernels]
        cat = self.dropout(cat)
        logits = self.classifier(cat)
        return logits

# 5.5 GCN Model (Standalone - Requires non-BERT input processing)
class GCNClassifier(nn.Module):
    # ... (Keep definition as is, ensure Dataset provides 'token_ids' and 'adj_matrix')
    def __init__(self, config):
        super().__init__()
        self.embedding = nn.Embedding(config['vocab_size'], config['embedding_dim'], padding_idx=0)
        self.dropout = nn.Dropout(config['dropout'])
        self.gcn_layers = nn.ModuleList()
        self.gcn_layers.append(GraphConvolution(config['embedding_dim'], config['hidden_dim']))
        for _ in range(1, config['gcn_layers']):
            self.gcn_layers.append(GraphConvolution(config['hidden_dim'], config['hidden_dim']))
        self.classifier = nn.Linear(config['hidden_dim'], config['num_classes'])

    def forward(self, token_ids, adj_matrix, **kwargs):
        # This expects 'token_ids' and 'adj_matrix' keys
        embedded = self.dropout(self.embedding(token_ids)) # [batch, seq_len, embed_dim]
        gcn_output = embedded
        for gcn_layer in self.gcn_layers:
            gcn_output = F.leaky_relu(gcn_layer(gcn_output, adj_matrix)) # [batch, seq_len, hidden_dim]
            gcn_output = self.dropout(gcn_output)

        # Pooling (Max or Mean)
        pooled = torch.max(gcn_output, dim=1)[0] # [batch, hidden_dim]
        # pooled = torch.mean(gcn_output, dim=1) # Alternative: Mean pooling

        logits = self.classifier(pooled)
        return logits

# 5.6 BERT-LSTM-GCN Model
class BERTBiLSTMGCNModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.bert_model_name = config['bert_model_name']
        self.bert_config = AutoConfig.from_pretrained(self.bert_model_name)
        self.bert = BertModel.from_pretrained(self.bert_model_name, config=self.bert_config)
        bert_dim = self.bert_config.hidden_size
        hidden_dim = config['hidden_dim']

        if config['freeze_bert']:
            print("Freezing BERT parameters.")
            for param in self.bert.parameters():
                param.requires_grad = False

        self.lstm = nn.LSTM(bert_dim, hidden_dim, config['lstm_layers'],
                            bidirectional=True, batch_first=True,
                            dropout=config['dropout'] if config['lstm_layers'] > 1 else 0)

        self.gcn_layers = nn.ModuleList()
        # GCN operates on BERT output dimension first
        self.gcn_layers.append(GraphConvolution(bert_dim, hidden_dim))
        for _ in range(1, config['gcn_layers']):
            self.gcn_layers.append(GraphConvolution(hidden_dim, hidden_dim))

        self.lstm_attention = nn.Linear(hidden_dim * 2, 1)
        self.lstm_proj = nn.Linear(hidden_dim * 2, hidden_dim)
        self.gcn_proj = nn.Linear(hidden_dim, hidden_dim) # GCN output is already hidden_dim

        self.use_linguistic = config['use_linguistic_features']
        if self.use_linguistic:
            self.linguistic_proj = nn.Linear(config['linguistic_feat_dim'], hidden_dim)
            fusion_dim = hidden_dim * 3 # LSTM + GCN + Ling
        else:
            fusion_dim = hidden_dim * 2 # LSTM + GCN

        self.fusion = nn.Linear(fusion_dim, hidden_dim)
        self.classifier = nn.Linear(hidden_dim, config['num_classes'])
        self.dropout = nn.Dropout(config['dropout'])

    def forward(self, input_ids, attention_mask, adj_matrix, linguistic_features=None, **kwargs):
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = bert_outputs.last_hidden_state # [batch, seq_len, bert_dim]
        sequence_output_dropout = self.dropout(sequence_output) # Apply dropout *once* after BERT

        # --- BiLSTM Path ---
        # Use the dropout applied version for consistency if needed, or original sequence_output
        lstm_output, _ = self.lstm(sequence_output_dropout) # [batch, seq_len, hidden*2]
        attn_weights = F.softmax(self.lstm_attention(lstm_output), dim=1)
        lstm_pooled = torch.sum(attn_weights * lstm_output, dim=1)
        lstm_features = F.leaky_relu(self.lstm_proj(lstm_pooled))
        lstm_features = self.dropout(lstm_features)

        # --- GCN Path ---
        # Input to GCN is BERT's sequence output
        gcn_input = sequence_output # Use original BERT output for GCN
        gcn_output = gcn_input
        for i, gcn_layer in enumerate(self.gcn_layers):
             # Apply dropout before activation for subsequent layers
             if i > 0:
                 gcn_output = self.dropout(gcn_output)
             gcn_output = F.leaky_relu(gcn_layer(gcn_output, adj_matrix))
        # Apply dropout after the last GCN layer before pooling
        gcn_output = self.dropout(gcn_output)

        # Max pooling for GCN
        gcn_pooled = torch.max(gcn_output, dim=1)[0]
        # GCN projection might not be needed if last GCN layer outputs hidden_dim
        # gcn_features = F.leaky_relu(self.gcn_proj(gcn_pooled)) # Optional projection
        gcn_features = gcn_pooled # Assume last GCN layer outputs hidden_dim
        gcn_features = self.dropout(gcn_features) # Dropout after pooling/projection

        # --- Combine Features ---
        combined_features = [lstm_features, gcn_features]

        if self.use_linguistic and linguistic_features is not None:
             # Ensure linguistic_features is float
             linguistic_features = linguistic_features.float()
             ling_features = F.leaky_relu(self.linguistic_proj(linguistic_features))
             ling_features = self.dropout(ling_features)
             combined_features.append(ling_features)

        combined = torch.cat(combined_features, dim=1)

        fused = F.leaky_relu(self.fusion(combined))
        fused = self.dropout(fused)
        logits = self.classifier(fused)

        return logits


# 5.7 BERT-LSTM-CNN Model
class BERTBiLSTMCnnModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.bert_model_name = config['bert_model_name']
        self.bert_config = AutoConfig.from_pretrained(self.bert_model_name)
        self.bert = BertModel.from_pretrained(self.bert_model_name, config=self.bert_config)
        bert_dim = self.bert_config.hidden_size
        hidden_dim = config['hidden_dim']

        if config['freeze_bert']:
            print("Freezing BERT parameters.")
            for param in self.bert.parameters():
                param.requires_grad = False

        self.lstm = nn.LSTM(bert_dim, hidden_dim, config['lstm_layers'],
                            bidirectional=True, batch_first=True,
                            dropout=config['dropout'] if config['lstm_layers'] > 1 else 0)

        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=bert_dim,
                      out_channels=config['cnn_filters'],
                      kernel_size=k)
            for k in config['cnn_kernel_sizes']
        ])
        cnn_output_dim = len(config['cnn_kernel_sizes']) * config['cnn_filters']

        self.lstm_attention = nn.Linear(hidden_dim * 2, 1)
        self.lstm_proj = nn.Linear(hidden_dim * 2, hidden_dim)
        self.cnn_proj = nn.Linear(cnn_output_dim, hidden_dim)

        self.use_linguistic = config['use_linguistic_features']
        if self.use_linguistic:
            self.linguistic_proj = nn.Linear(config['linguistic_feat_dim'], hidden_dim)
            fusion_dim = hidden_dim * 3 # LSTM + CNN + Ling
        else:
            fusion_dim = hidden_dim * 2 # LSTM + CNN

        self.fusion = nn.Linear(fusion_dim, hidden_dim)
        self.classifier = nn.Linear(hidden_dim, config['num_classes'])
        self.dropout = nn.Dropout(config['dropout'])

    def forward(self, input_ids, attention_mask, linguistic_features=None, **kwargs): # adj_matrix not needed
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = bert_outputs.last_hidden_state # [batch, seq_len, bert_dim]
        sequence_output_dropout = self.dropout(sequence_output)

        # --- BiLSTM Path ---
        lstm_output, _ = self.lstm(sequence_output_dropout)
        attn_weights = F.softmax(self.lstm_attention(lstm_output), dim=1)
        lstm_pooled = torch.sum(attn_weights * lstm_output, dim=1)
        lstm_features = F.leaky_relu(self.lstm_proj(lstm_pooled))
        lstm_features = self.dropout(lstm_features)

        # --- CNN Path ---
        # Input needs to be [batch, bert_dim, seq_len]
        cnn_input = sequence_output.permute(0, 2, 1)
        # Apply dropout before CNN? Or rely on dropout after BERT? Let's apply after BERT only.
        # cnn_input_dropout = sequence_output_dropout.permute(0, 2, 1) # Option
        conved = [F.leaky_relu(conv(cnn_input)) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        cnn_pooled = torch.cat(pooled, dim=1)
        # Apply dropout before projection
        cnn_pooled = self.dropout(cnn_pooled)
        cnn_features = F.leaky_relu(self.cnn_proj(cnn_pooled))
        # Apply dropout after projection? Let's stick to after pooling/before projection.

        # --- Combine Features ---
        combined_features = [lstm_features, cnn_features]

        if self.use_linguistic and linguistic_features is not None:
            linguistic_features = linguistic_features.float() # Ensure float
            ling_features = F.leaky_relu(self.linguistic_proj(linguistic_features))
            ling_features = self.dropout(ling_features)
            combined_features.append(ling_features)

        combined = torch.cat(combined_features, dim=1)

        fused = F.leaky_relu(self.fusion(combined))
        fused = self.dropout(fused)
        logits = self.classifier(fused)

        return logits

## Helper Functions

In [12]:
# --- 6. Helper Functions ---

def get_model(config):
    """Factory function to create model instance based on config."""
    model_type = config['model_type']
    print(f"Initializing model: {model_type}")

    if model_type == 'bert':
        model = BERTClassifier(config)
    elif model_type == 'lstm':
        model = LSTMClassifier(config) # Requires Dataset providing 'token_ids'
    elif model_type == 'cnn':
        model = CNNClassifier(config)  # Requires Dataset providing 'token_ids'
    elif model_type == 'gcn':
        model = GCNClassifier(config) # Requires Dataset providing 'token_ids', 'adj_matrix'
    elif model_type == 'bert_lstm_gcn':
        model = BERTBiLSTMGCNModel(config)
    elif model_type == 'bert_lstm_cnn':
        model = BERTBiLSTMCnnModel(config)
    else:
        raise ValueError(f"Unknown model_type: {model_type}")

    # Move model to the primary device specified in config
    return model.to(config['device'])

def visualize_model(model, dataloader, config, filename="model_graph"):
    """Visualizes the model using torchviz."""
    if not torchviz:
        print("torchviz not installed. Skipping model visualization.")
        return
    # ===> IMPORTANT: Ensure visualization happens BEFORE DataParallel wrapping <===
    if isinstance(model, nn.DataParallel):
        print("Warning: Visualizing DataParallel wrapped model. Visualizing the underlying module.")
        model_to_viz = model.module
    else:
        model_to_viz = model

    try:
        sample_batch = next(iter(dataloader))
        # Move only necessary tensor inputs to device
        inputs = {k: v.to(config['device']) for k, v in sample_batch.items()
                  if isinstance(v, torch.Tensor) and k != 'label'} # Exclude label

        # Ensure inputs match the model's forward signature (after potential DP wrapping)
        # Filter inputs based on the actual forward signature (more robust)
        # This part is complex; simpler to assume inputs are correct for now.

        y = model_to_viz(**inputs)

        dot = torchviz.make_dot(y, params=dict(model_to_viz.named_parameters()))
        dot.format = 'png'
        dot.render(filename)
        print(f"Model graph saved to {filename}.png")
    except Exception as e:
        print(f"Could not visualize model: {e}")
        print("Check if sample batch inputs match the model's forward signature.")
        import traceback
        traceback.print_exc()

## Training & Evaluation Logic

In [13]:
# --- 7. Training & Evaluation Logic ---

def train_epoch(model, dataloader, optimizer, criterion, device, config, epoch):
    model.train() # Set model to training mode
    epoch_loss = 0
    correct = 0
    total = 0

    progress_bar = tqdm(dataloader, desc="Training", leave=False, dynamic_ncols=True)
    for batch in progress_bar:
        # Move tensors to the primary device (DataParallel handles distribution)
        inputs = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
        labels = inputs.pop('label')

        optimizer.zero_grad()

        # Forward pass - DataParallel handles splitting/gathering
        outputs = model(**inputs)

        # Loss calculation happens on the primary device (where outputs are gathered)
        loss = criterion(outputs, labels)

        # Check for NaN loss
        if torch.isnan(loss):
             print("\nNaN loss detected! Skipping batch.")
             # Optionally add more debugging here: print inputs, outputs etc.
             # Consider gradient clipping if gradients explode.
             torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # Example clipping
             continue # Skip backward pass for this batch


        # Backward pass - Gradients are computed on each GPU and summed on primary
        loss.backward()

        # Optional: Gradient Clipping (useful if gradients explode)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        # Metrics calculated on primary device
        epoch_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        batch_acc = (predicted == labels).sum().item() / labels.size(0) if labels.size(0) > 0 else 0
        progress_bar.set_postfix({'loss': f"{loss.item():.4f}", 'acc': f"{batch_acc:.4f}"})

    if total == 0: return 0.0, 0.0 # Avoid division by zero
    accuracy = correct / total
    avg_loss = epoch_loss / len(dataloader)
    return avg_loss, accuracy

def evaluate_epoch(model, dataloader, criterion, device, config):
    model.eval() # Set model to evaluation mode
    epoch_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    all_probs = []

    progress_bar = tqdm(dataloader, desc="Evaluating", leave=False, dynamic_ncols=True)
    with torch.no_grad():
        for batch in progress_bar:
            inputs = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
            labels = inputs.pop('label')

            outputs = model(**inputs)
            loss = criterion(outputs, labels) # Calculated on primary GPU

            epoch_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Collect predictions and labels (move to CPU for numpy/sklearn)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

            if outputs.shape[1] >= 2:
                probs = F.softmax(outputs, dim=1)[:, 1].cpu().numpy() # Prob for positive class
                all_probs.extend(probs)
            elif outputs.shape[1] == 1:
                 probs = torch.sigmoid(outputs).squeeze().cpu().numpy()
                 all_probs.extend(probs)


    if total == 0: return 0.0, 0.0, None, [], [] # Avoid division by zero
    accuracy = correct / total
    avg_loss = epoch_loss / len(dataloader)

    roc_auc = None
    # Ensure binary classification and probabilities were collected
    if len(np.unique(all_labels)) == 2 and len(all_probs) == len(all_labels):
        try:
            # Ensure labels are binary (0, 1) for roc_auc_score
            # The LabelEncoder already produced 0 and 1.
            roc_auc = roc_auc_score(all_labels, all_probs)
        except ValueError as e:
            print(f"Warning: Could not calculate ROC AUC - {e}")
        except Exception as e: # Catch other potential errors
            print(f"Warning: Error calculating ROC AUC: {e}")


    return avg_loss, accuracy, roc_auc, all_labels, all_preds

def plot_training_history(history, filename="training_history.png"):
    """Plots loss and accuracy curves."""
    epochs_ran = len(history['train_loss'])
    if epochs_ran == 0:
        print("No history to plot.")
        return
    epochs = range(1, epochs_ran + 1)

    plt.figure(figsize=(12, 5))

    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], 'bo-', label='Training Loss')
    plt.plot(epochs, history['val_loss'], 'ro-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['train_acc'], 'bo-', label='Training Accuracy')
    plt.plot(epochs, history['val_acc'], 'ro-', label='Validation Accuracy')
    # Filter out None values before plotting ROC AUC
    valid_roc_auc = [(e, auc) for e, auc in zip(epochs, history.get('val_roc_auc', [])) if auc is not None]
    if valid_roc_auc:
        roc_epochs, roc_values = zip(*valid_roc_auc)
        plt.plot(roc_epochs, roc_values, 'go-', label='Validation ROC AUC')

    plt.title('Training and Validation Metrics')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy / ROC AUC')
    # Adjust ylim if needed, e.g., plt.ylim(0, 1.05) for accuracy/AUC
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig(filename)
    print(f"Training history plot saved to {filename}")
    plt.close() # Close the plot to free memory, especially in loops/notebooks

def save_training_log(history, filename):
    """Saves training history to a JSON file."""
    try:
        # Convert numpy arrays or tensors in history to lists for JSON serialization
        serializable_history = {}
        for key, value in history.items():
            if isinstance(value, list) and value and isinstance(value[0], (np.ndarray, torch.Tensor)):
                 serializable_history[key] = [item.tolist() if hasattr(item, 'tolist') else item for item in value]
            elif isinstance(value, (np.ndarray, torch.Tensor)):
                 serializable_history[key] = value.tolist()
            else:
                 serializable_history[key] = value # Assume already serializable

        with open(filename, 'w') as f:
            json.dump(serializable_history, f, indent=4)
        print(f"Training log saved to {filename}")
    except Exception as e:
        print(f"Error saving training log: {e}")




def run_training(config, model, train_dataloader, val_dataloader):
    """Main training loop."""
    device = config['device'] # Primary device
    lr = config['learning_rate'] if 'bert' in config['model_type'] else config['non_bert_lr']
    # Pass model.module.parameters() if DataParallel, else model.parameters()
    # Optimizer usually handles DataParallel correctly, passing model.parameters() is fine.
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    best_val_metric = -1
    metric_to_monitor = 'acc' # 'acc', 'roc_auc', 'loss'
    patience_counter = 0
    patience_limit = 3 # Example early stopping patience

    training_history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': [], 'val_roc_auc': []
    }

    print("\n--- Starting Training ---")
    num_gpus = torch.cuda.device_count()
    model_is_parallel = isinstance(model, nn.DataParallel)
    if model_is_parallel:
        print(f"Training with DataParallel on {num_gpus} GPUs.")
    elif device != 'cpu':
         print(f"Training on single GPU: {torch.cuda.get_device_name(0)}")
    else:
        print("Training on CPU.")


    for epoch in range(config['epochs']):
        print(f"\nEpoch {epoch+1}/{config['epochs']}")

        train_loss, train_acc = train_epoch(model, train_dataloader, optimizer, criterion, device, config, epoch + 1) # Pass epoch number
        training_history['train_loss'].append(train_loss)
        training_history['train_acc'].append(train_acc)

        val_loss, val_acc, val_roc_auc, _, _ = evaluate_epoch(model, val_dataloader, criterion, device, config)
        training_history['val_loss'].append(val_loss)
        training_history['val_acc'].append(val_acc)
        training_history['val_roc_auc'].append(val_roc_auc) # Can be None

        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}", end="")
        if val_roc_auc is not None:
            print(f", Val ROC AUC: {val_roc_auc:.4f}")
        else:
            print(", Val ROC AUC: N/A") # Handle None case

        # === WANDB INTEGRATION START ===
        # --- Log metrics to WandB ---
        log_dict = {
            "epoch": epoch + 1,
            "train/loss": train_loss,
            "train/accuracy": train_acc,
            "val/loss": val_loss,
            "val/accuracy": val_acc,
        }
        # Only log ROC AUC if it's calculated
        if val_roc_auc is not None:
            log_dict["val/roc_auc"] = val_roc_auc
        wandb.log(log_dict)
        # === WANDB INTEGRATION END ===

        # Determine the current metric value to check for improvement
        current_val_metric = None
        if metric_to_monitor == 'acc':
            current_val_metric = val_acc
        elif metric_to_monitor == 'roc_auc':
            current_val_metric = val_roc_auc # Could be None
        elif metric_to_monitor == 'loss':
             current_val_metric = val_loss # Lower is better

        is_better = False
        if current_val_metric is not None: # Only compare if the metric is valid
            if metric_to_monitor == 'loss':
                is_better = current_val_metric < best_val_metric if best_val_metric != float('inf') else True
                if is_better: best_val_metric = current_val_metric # Update best_val_metric if lower loss
            else: # Accuracy or ROC AUC - higher is better
                is_better = current_val_metric > best_val_metric
                if is_better: best_val_metric = current_val_metric # Update best_val_metric if higher metric


        if is_better:
            print(f"Validation {metric_to_monitor} improved ({best_val_metric:.4f} -> {current_val_metric:.4f}). Saving model...")
            best_val_metric = current_val_metric
            try:
                # ====> SAVE CORRECT STATE DICT <====
                if isinstance(model, nn.DataParallel):
                    model_state = model.module.state_dict() # Save the underlying model's state
                else:
                    model_state = model.state_dict()
                save_file(model_state, config['model_save_path'])
                print(f"Best model saved to {config['model_save_path']}")

                # === WANDB INTEGRATION START ===
                # --- Log the best model as an artifact ---
                try:
                    model_artifact_name = f"{wandb.run.name}-best-model"
                    config_dict = dict(config)
                    artifact = wandb.Artifact(model_artifact_name, type='model', metadata=config_dict)
                    artifact.add_file(config['model_save_path'])
                    wandb.log_artifact(artifact, aliases=["best", f"epoch_{epoch+1}"])
                    print(f"Wandb: Logged best model artifact: {model_artifact_name}")
                except Exception as e:
                    print(f"Wandb: Error logging model artifact: {e}")
                # === WANDB INTEGRATION END ===
                
                patience_counter = 0 # Reset patience
            except Exception as e:
                print(f"Error saving model: {e}")
        else:
             patience_counter += 1
             print(f"Validation {metric_to_monitor} did not improve. Patience: {patience_counter}/{patience_limit}")
             if patience_counter >= patience_limit:
                 print("Early stopping triggered.")
                 break # Stop training


    print("--- Training Finished ---")
    save_training_log(training_history, config['log_file'])
    plot_training_history(training_history) # Plot at the end

    # Return the model (it might be wrapped in DataParallel) and history

    # === WANDB INTEGRATION START ===
    # --- Log the final training history plot ---
    try:
        history_plot_filename = "training_history.png"
        if os.path.exists(history_plot_filename):
            wandb.log({"training_curves": wandb.Image(history_plot_filename)})
    except Exception as e:
        print(f"Wandb: Could not log training history plot: {e}")
    # === WANDB INTEGRATION END ===

    
    return model, training_history

In [14]:
def evaluate_model_on_test(config, test_dataloader, label_encoder):
    """Evaluates the final *saved* model on the test set."""
    print("\n--- Evaluating on Test Set ---")
    device = config['device']
    criterion = nn.CrossEntropyLoss() # For loss calculation only

    # 1. Create a fresh model instance on the primary device
    model = get_model(config)

    # 2. Load the *base model* state dict saved during training
    try:
        print(f"Loading best model state from: {config['model_save_path']}")
        # Load state dict onto the primary device directly
        state_dict = load_file(config['model_save_path'], device=device)
        model.load_state_dict(state_dict)
        print("Model state loaded successfully.")
    except FileNotFoundError:
        print(f"Error: Model file not found at {config['model_save_path']}. Cannot evaluate.")
        # === WANDB INTEGRATION START ===
        if wandb.run: # Check if a wandb run is active before finishing
             wandb.finish()
        # === WANDB INTEGRATION END ===
        return
    except Exception as e:
        print(f"Error loading model state: {e}. Cannot evaluate.")
        # === WANDB INTEGRATION START ===
        if wandb.run:
             wandb.finish()
        # === WANDB INTEGRATION END ===
        return

    # 3. Wrap the loaded model with DataParallel if multiple GPUs are available for evaluation
    num_gpus = torch.cuda.device_count()
    if num_gpus > 1 and device != 'cpu':
        print(f"Using {num_gpus} GPUs for evaluation! Wrapping model with DataParallel.")
        model = nn.DataParallel(model)
        # Note: model is already on the primary device 'cuda:0' from get_model

    model.to(device) # Ensure model is on the correct device (redundant if already done, but safe)

    # 4. Perform evaluation
    test_loss, test_acc, test_roc_auc, y_true, y_pred = evaluate_epoch(model, test_dataloader, criterion, device, config)

    print(f"\nTest Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    if test_roc_auc is not None:
        print(f"Test ROC AUC: {test_roc_auc:.4f}")
    else:
        print("Test ROC AUC: N/A")


    # === WANDB INTEGRATION START ===
    # --- Log final test metrics ---
    # Use wandb.summary for metrics that summarize the *entire run*
    if wandb.run: # Check if run is active
        wandb.summary["test/loss"] = test_loss
        wandb.summary["test/accuracy"] = test_acc
        if test_roc_auc is not None:
            wandb.summary["test/roc_auc"] = test_roc_auc
    # === WANDB INTEGRATION END ===


    # Ensure labels and preds are available before generating reports
    if not y_true or not y_pred:
         print("Evaluation failed, no predictions generated.")
         # === WANDB INTEGRATION START ===
         # Finish the run even if evaluation fails partially
         if wandb.run:
             wandb.finish()
         # === WANDB INTEGRATION END ===
         return


    # Classification Report
    target_names = label_encoder.classes_.astype(str)
    print("\nClassification Report:")
    try: # Outer try for classification report generation/logging
        # Ensure y_true and y_pred are numpy arrays
        y_true_np = np.array(y_true)
        y_pred_np = np.array(y_pred)
        report_str = classification_report(y_true_np, y_pred_np, target_names=target_names, digits=4)
        print(report_str)
        # === WANDB INTEGRATION START ===
        # Log classification report as text (optional)
        try: # Inner try for logging
             if wandb.run:
                 wandb.log({"test/classification_report": wandb.Html(f"<pre>{report_str}</pre>")})
        except Exception as e:
             print(f"Wandb: Error logging classification report: {e}")
        # === WANDB INTEGRATION END ===
    except Exception as e: # <--- **** ADDED THIS EXCEPT BLOCK ****
         print(f"Could not generate or log classification report: {e}")
    # <--- **** END OF ADDED BLOCK **** ---

    # Confusion Matrix
    print("\nConfusion Matrix:")
    try: # Outer try for overall CM generation/saving
        y_true_np = np.array(y_true)
        y_pred_np = np.array(y_pred)
        cm = confusion_matrix(y_true_np, y_pred_np)
        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.title('Confusion Matrix (Test Set)')
        cm_filename = "confusion_matrix_test.png"
        plt.savefig(cm_filename)
        print(f"Confusion matrix saved to {cm_filename}")
        plt.close() # Close plot

        # === WANDB INTEGRATION START ===
        # --- Log confusion matrix plot ---
        try: # Inner try specifically for logging the plot
            if wandb.run:
                wandb.log({"test/confusion_matrix": wandb.Image(cm_filename)})
        except Exception as e: # Exception for the INNER try (wandb logging)
            print(f"Wandb: Could not log confusion matrix image: {e}")
        # === WANDB INTEGRATION END ===

    except Exception as e: # Exception for the OUTER try (generating/saving the plot)
        print(f"Could not generate or save confusion matrix plot: {e}")

# Main Execution

## CNN

In [18]:
if __name__ == "__main__":
    # === WANDB INTEGRATION START ===
    # --- Log in to WandB using Kaggle Secrets ---
    try:
        from kaggle_secrets import UserSecretsClient
        import os # Import os to potentially set environment variable

        user_secrets = UserSecretsClient()

        # Retrieve the secret variable (ensure the label matches what you set in Kaggle Secrets)
        # METHOD 1: Directly use the key with wandb.login (Recommended for clarity)
        wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")
        wandb.login(key=wandb_api_key)
        print("Wandb login successful using Kaggle Secret.")

        # METHOD 2: Set environment variable (wandb.init often picks this up automatically)
        # os.environ["WANDB_API_KEY"] = user_secrets.get_secret("WANDB_API_KEY")
        # print("WANDB_API_KEY environment variable set from Kaggle Secret.")
        # If using Method 2, wandb.login() might not even be strictly necessary
        # as wandb.init() usually checks this environment variable.

    except ImportError:
        print("kaggle_secrets not found, assuming API key is set elsewhere (e.g., environment variable).")
    except Exception as e:
        print(f"Error retrieving WANDB_API_KEY from Kaggle Secrets or logging in: {e}")
        # Decide how to handle this - maybe exit, maybe proceed if key might be elsewhere
        # For now, we'll proceed, but wandb logging might fail.
        pass # Or raise e if login is critical


    # --- Initialize WandB Run ---
    run_name = f"{config['model_type']}-e{config['epochs']}-bs{config['batch_size']}-lr{config['learning_rate']}"
    # wandb.init should now work without prompting if login was successful or env var is set
    wandb.init(
        project="sentiment-analysis-hybrid",  # Replace with your project name
        config=config,
        name=run_name,
        job_type="train",
    )
    config = wandb.config # Copy back config
    # === WANDB INTEGRATION END ===
    
    # 1. Get the base model instance (moved to primary device by get_model)
    model = get_model(config)
    primary_device = config['device']
    num_gpus = torch.cuda.device_count()

    # === WANDB INTEGRATION START ===
    # --- Watch the model (optional but useful for gradients/parameters) ---
    # Apply wandb.watch *before* DataParallel wrapping if possible
    # Note: Watching DataParallel models might require watching model.module later
    try:
        # Log gradients and parameters of the model
        wandb.watch(model, log="all", log_freq=100) # Log every 100 batches
    except Exception as e:
        print(f"Could not watch model with wandb: {e}")
    # === WANDB INTEGRATION END ===

    # 2. (Optional) Visualize Architecture *before* wrapping
    if torchviz and len(train_dataloader) > 0:
        print("Attempting model visualization...")
        # Pass the unwrapped model
        visualize_model(model, train_dataloader, config, filename=f"{config['model_type']}_graph")
       
        # === WANDB INTEGRATION START ===
        # Log the model graph image (if created)
        try:
            graph_filename = f"{config['model_type']}_graph.png"
            if os.path.exists(graph_filename):
                 wandb.log({"model_architecture": wandb.Image(graph_filename)})
        except Exception as e:
            print(f"Wandb: Could not log model architecture image: {e}")
        # === WANDB INTEGRATION END ===

    # 3. Wrap model with DataParallel *before* training if multiple GPUs available
    if num_gpus > 1 and primary_device != 'cpu':
        print(f"Wrapping model with nn.DataParallel for training across {num_gpus} GPUs.")
        model = nn.DataParallel(model)
        # DataParallel will manage distribution to devices cuda:0, cuda:1, ...
        # Model should already be on cuda:0 from get_model

    # Ensure model is on the correct device (important if CPU or single GPU)
    model.to(primary_device)

    # 4. Latih Model (pass the potentially wrapped model)
    # run_training now returns the potentially wrapped model and history
    trained_model, history = run_training(config, model, train_dataloader, val_dataloader)

    # 5. Evaluasi Model Terbaik di Test Set
    # The evaluation function now handles loading the saved *base* state dict
    # and potentially wrapping the model again for evaluation.
    evaluate_model_on_test(config, test_dataloader, label_encoder)

    print("\n--- Analysis Complete ---")

    # === WANDB INTEGRATION START ===
    # --- Finish the WandB run ---
    wandb.finish()
    # === WANDB INTEGRATION END ===


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: maybeitsai (hkacode). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Wandb login successful using Kaggle Secret.


Initializing model: cnn
Attempting model visualization...
Model graph saved to cnn_graph.png
Wrapping model with nn.DataParallel for training across 2 GPUs.

--- Starting Training ---
Training with DataParallel on 2 GPUs.

Epoch 1/3


Training:   0%|          | 0/782 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/157 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/157 [00:00<?, ?it/s]

Train Loss: 0.3095, Train Acc: 0.8702
Val Loss: 0.3259, Val Acc: 0.8590, Val ROC AUC: 0.9359
Validation acc improved (0.8590 -> 0.8590). Saving model...
Best model saved to best_model.safetensors
Wandb: Logged best model artifact: cnn-e3-bs128-lr2e-05-best-model
--- Training Finished ---
Training log saved to training_log.json
Training history plot saved to training_history.png

--- Evaluating on Test Set ---
Initializing model: cnn
Loading best model state from: best_model.safetensors
Model state loaded successfully.
Using 2 GPUs for evaluation! Wrapping model with DataParallel.


Evaluating:   0%|          | 0/235 [00:00<?, ?it/s]


Test Loss: 0.3173
Test Accuracy: 0.8602
Test ROC AUC: 0.9386

Classification Report:
              precision    recall  f1-score   support

    negative     0.8655    0.8530    0.8592     15000
    positive     0.8551    0.8674    0.8612     15000

    accuracy                         0.8602     30000
   macro avg     0.8603    0.8602    0.8602     30000
weighted avg     0.8603    0.8602    0.8602     30000


Confusion Matrix:
Confusion matrix saved to confusion_matrix_test.png

--- Analysis Complete ---


epoch,▁▅█
train/accuracy,▁▆█
train/loss,█▃▁
val/accuracy,▁▄█
val/loss,█▆▁
val/roc_auc,▁▆█
epoch,3
test/accuracy,0.8602
test/loss,0.31728
test/roc_auc,0.93856
train/accuracy,0.87023


## LSTM

In [15]:
if __name__ == "__main__":
    # === WANDB INTEGRATION START ===
    # --- Log in to WandB using Kaggle Secrets ---
    try:
        from kaggle_secrets import UserSecretsClient
        import os # Import os to potentially set environment variable

        user_secrets = UserSecretsClient()

        # Retrieve the secret variable (ensure the label matches what you set in Kaggle Secrets)
        # METHOD 1: Directly use the key with wandb.login (Recommended for clarity)
        wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")
        wandb.login(key=wandb_api_key)
        print("Wandb login successful using Kaggle Secret.")

        # METHOD 2: Set environment variable (wandb.init often picks this up automatically)
        # os.environ["WANDB_API_KEY"] = user_secrets.get_secret("WANDB_API_KEY")
        # print("WANDB_API_KEY environment variable set from Kaggle Secret.")
        # If using Method 2, wandb.login() might not even be strictly necessary
        # as wandb.init() usually checks this environment variable.

    except ImportError:
        print("kaggle_secrets not found, assuming API key is set elsewhere (e.g., environment variable).")
    except Exception as e:
        print(f"Error retrieving WANDB_API_KEY from Kaggle Secrets or logging in: {e}")
        # Decide how to handle this - maybe exit, maybe proceed if key might be elsewhere
        # For now, we'll proceed, but wandb logging might fail.
        pass # Or raise e if login is critical


    # --- Initialize WandB Run ---
    run_name = f"{config['model_type']}-e{config['epochs']}-bs{config['batch_size']}-lr{config['learning_rate']}"
    # wandb.init should now work without prompting if login was successful or env var is set
    wandb.init(
        project="sentiment-analysis-hybrid",  # Replace with your project name
        config=config,
        name=run_name,
        job_type="train",
    )
    config = wandb.config # Copy back config
    # === WANDB INTEGRATION END ===
    
    # 1. Get the base model instance (moved to primary device by get_model)
    model = get_model(config)
    primary_device = config['device']
    num_gpus = torch.cuda.device_count()

    # === WANDB INTEGRATION START ===
    # --- Watch the model (optional but useful for gradients/parameters) ---
    # Apply wandb.watch *before* DataParallel wrapping if possible
    # Note: Watching DataParallel models might require watching model.module later
    try:
        # Log gradients and parameters of the model
        wandb.watch(model, log="all", log_freq=100) # Log every 100 batches
    except Exception as e:
        print(f"Could not watch model with wandb: {e}")
    # === WANDB INTEGRATION END ===

    # 2. (Optional) Visualize Architecture *before* wrapping
    if torchviz and len(train_dataloader) > 0:
        print("Attempting model visualization...")
        # Pass the unwrapped model
        visualize_model(model, train_dataloader, config, filename=f"{config['model_type']}_graph")
       
        # === WANDB INTEGRATION START ===
        # Log the model graph image (if created)
        try:
            graph_filename = f"{config['model_type']}_graph.png"
            if os.path.exists(graph_filename):
                 wandb.log({"model_architecture": wandb.Image(graph_filename)})
        except Exception as e:
            print(f"Wandb: Could not log model architecture image: {e}")
        # === WANDB INTEGRATION END ===

    # 3. Wrap model with DataParallel *before* training if multiple GPUs available
    if num_gpus > 1 and primary_device != 'cpu':
        print(f"Wrapping model with nn.DataParallel for training across {num_gpus} GPUs.")
        model = nn.DataParallel(model)
        # DataParallel will manage distribution to devices cuda:0, cuda:1, ...
        # Model should already be on cuda:0 from get_model

    # Ensure model is on the correct device (important if CPU or single GPU)
    model.to(primary_device)

    # 4. Latih Model (pass the potentially wrapped model)
    # run_training now returns the potentially wrapped model and history
    trained_model, history = run_training(config, model, train_dataloader, val_dataloader)

    # 5. Evaluasi Model Terbaik di Test Set
    # The evaluation function now handles loading the saved *base* state dict
    # and potentially wrapping the model again for evaluation.
    evaluate_model_on_test(config, test_dataloader, label_encoder)

    print("\n--- Analysis Complete ---")

    # === WANDB INTEGRATION START ===
    # --- Finish the WandB run ---
    wandb.finish()
    # === WANDB INTEGRATION END ===


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: maybeitsai (hkacode). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Wandb login successful using Kaggle Secret.


Initializing model: lstm
Attempting model visualization...
Model graph saved to lstm_graph.png
Wrapping model with nn.DataParallel for training across 2 GPUs.

--- Starting Training ---
Training with DataParallel on 2 GPUs.

Epoch 1/3


Training:   0%|          | 0/782 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/157 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/157 [00:00<?, ?it/s]

Train Loss: 0.2407, Train Acc: 0.9036
Val Loss: 0.4938, Val Acc: 0.7783, Val ROC AUC: 0.9407
Validation acc did not improve. Patience: 1/3
--- Training Finished ---
Training log saved to training_log.json
Training history plot saved to training_history.png

--- Evaluating on Test Set ---
Initializing model: lstm
Loading best model state from: best_model.safetensors
Model state loaded successfully.
Using 2 GPUs for evaluation! Wrapping model with DataParallel.


Evaluating:   0%|          | 0/235 [00:00<?, ?it/s]


Test Loss: 0.3602
Test Accuracy: 0.8287
Test ROC AUC: 0.9393

Classification Report:
              precision    recall  f1-score   support

    negative     0.9417    0.7007    0.8035     15000
    positive     0.7617    0.9566    0.8481     15000

    accuracy                         0.8287     30000
   macro avg     0.8517    0.8287    0.8258     30000
weighted avg     0.8517    0.8287    0.8258     30000


Confusion Matrix:
Confusion matrix saved to confusion_matrix_test.png

--- Analysis Complete ---


epoch,▁▅█
train/accuracy,▁▇█
train/loss,█▃▁
val/accuracy,▁█▁
val/loss,▇▁█
val/roc_auc,▁▇█
epoch,3
test/accuracy,0.82867
test/loss,0.36022
test/roc_auc,0.93929
train/accuracy,0.90358


## GCN

In [16]:
if __name__ == "__main__":
    # === WANDB INTEGRATION START ===
    # --- Log in to WandB using Kaggle Secrets ---
    try:
        from kaggle_secrets import UserSecretsClient
        import os # Import os to potentially set environment variable

        user_secrets = UserSecretsClient()

        # Retrieve the secret variable (ensure the label matches what you set in Kaggle Secrets)
        # METHOD 1: Directly use the key with wandb.login (Recommended for clarity)
        wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")
        wandb.login(key=wandb_api_key)
        print("Wandb login successful using Kaggle Secret.")

        # METHOD 2: Set environment variable (wandb.init often picks this up automatically)
        # os.environ["WANDB_API_KEY"] = user_secrets.get_secret("WANDB_API_KEY")
        # print("WANDB_API_KEY environment variable set from Kaggle Secret.")
        # If using Method 2, wandb.login() might not even be strictly necessary
        # as wandb.init() usually checks this environment variable.

    except ImportError:
        print("kaggle_secrets not found, assuming API key is set elsewhere (e.g., environment variable).")
    except Exception as e:
        print(f"Error retrieving WANDB_API_KEY from Kaggle Secrets or logging in: {e}")
        # Decide how to handle this - maybe exit, maybe proceed if key might be elsewhere
        # For now, we'll proceed, but wandb logging might fail.
        pass # Or raise e if login is critical


    # --- Initialize WandB Run ---
    run_name = f"{config['model_type']}-e{config['epochs']}-bs{config['batch_size']}-lr{config['learning_rate']}"
    # wandb.init should now work without prompting if login was successful or env var is set
    wandb.init(
        project="sentiment-analysis-hybrid",  # Replace with your project name
        config=config,
        name=run_name,
        job_type="train",
    )
    config = wandb.config # Copy back config
    # === WANDB INTEGRATION END ===
    
    # 1. Get the base model instance (moved to primary device by get_model)
    model = get_model(config)
    primary_device = config['device']
    num_gpus = torch.cuda.device_count()

    # === WANDB INTEGRATION START ===
    # --- Watch the model (optional but useful for gradients/parameters) ---
    # Apply wandb.watch *before* DataParallel wrapping if possible
    # Note: Watching DataParallel models might require watching model.module later
    try:
        # Log gradients and parameters of the model
        wandb.watch(model, log="all", log_freq=100) # Log every 100 batches
    except Exception as e:
        print(f"Could not watch model with wandb: {e}")
    # === WANDB INTEGRATION END ===

    # 2. (Optional) Visualize Architecture *before* wrapping
    if torchviz and len(train_dataloader) > 0:
        print("Attempting model visualization...")
        # Pass the unwrapped model
        visualize_model(model, train_dataloader, config, filename=f"{config['model_type']}_graph")
       
        # === WANDB INTEGRATION START ===
        # Log the model graph image (if created)
        try:
            graph_filename = f"{config['model_type']}_graph.png"
            if os.path.exists(graph_filename):
                 wandb.log({"model_architecture": wandb.Image(graph_filename)})
        except Exception as e:
            print(f"Wandb: Could not log model architecture image: {e}")
        # === WANDB INTEGRATION END ===

    # 3. Wrap model with DataParallel *before* training if multiple GPUs available
    if num_gpus > 1 and primary_device != 'cpu':
        print(f"Wrapping model with nn.DataParallel for training across {num_gpus} GPUs.")
        model = nn.DataParallel(model)
        # DataParallel will manage distribution to devices cuda:0, cuda:1, ...
        # Model should already be on cuda:0 from get_model

    # Ensure model is on the correct device (important if CPU or single GPU)
    model.to(primary_device)

    # 4. Latih Model (pass the potentially wrapped model)
    # run_training now returns the potentially wrapped model and history
    trained_model, history = run_training(config, model, train_dataloader, val_dataloader)

    # 5. Evaluasi Model Terbaik di Test Set
    # The evaluation function now handles loading the saved *base* state dict
    # and potentially wrapping the model again for evaluation.
    evaluate_model_on_test(config, test_dataloader, label_encoder)

    print("\n--- Analysis Complete ---")

    # === WANDB INTEGRATION START ===
    # --- Finish the WandB run ---
    wandb.finish()
    # === WANDB INTEGRATION END ===


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: maybeitsai (hkacode). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Wandb login successful using Kaggle Secret.


Initializing model: gcn
Attempting model visualization...
Model graph saved to gcn_graph.png
Wrapping model with nn.DataParallel for training across 2 GPUs.

--- Starting Training ---
Training with DataParallel on 2 GPUs.

Epoch 1/3


Training:   0%|          | 0/782 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/157 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/157 [00:00<?, ?it/s]

Train Loss: 0.2916, Train Acc: 0.8765
Val Loss: 0.3611, Val Acc: 0.8470, Val ROC AUC: 0.9288
Validation acc improved (0.8470 -> 0.8470). Saving model...
Best model saved to best_model.safetensors
Wandb: Logged best model artifact: gcn-e3-bs128-lr2e-05-best-model
--- Training Finished ---
Training log saved to training_log.json
Training history plot saved to training_history.png

--- Evaluating on Test Set ---
Initializing model: gcn
Loading best model state from: best_model.safetensors
Model state loaded successfully.
Using 2 GPUs for evaluation! Wrapping model with DataParallel.


Evaluating:   0%|          | 0/235 [00:00<?, ?it/s]


Test Loss: 0.3587
Test Accuracy: 0.8499
Test ROC AUC: 0.9302

Classification Report:
              precision    recall  f1-score   support

    negative     0.9003    0.7869    0.8398     15000
    positive     0.8107    0.9129    0.8588     15000

    accuracy                         0.8499     30000
   macro avg     0.8555    0.8499    0.8493     30000
weighted avg     0.8555    0.8499    0.8493     30000


Confusion Matrix:
Confusion matrix saved to confusion_matrix_test.png

--- Analysis Complete ---


epoch,▁▅█
train/accuracy,▁▇█
train/loss,█▃▁
val/accuracy,▁▆█
val/loss,█▃▁
val/roc_auc,▁▅█
epoch,3
test/accuracy,0.84987
test/loss,0.35874
test/roc_auc,0.93017
train/accuracy,0.87649


## BERT

In [16]:
if __name__ == "__main__":
    # === WANDB INTEGRATION START ===
    # --- Log in to WandB using Kaggle Secrets ---
    try:
        from kaggle_secrets import UserSecretsClient
        import os # Import os to potentially set environment variable

        user_secrets = UserSecretsClient()

        # Retrieve the secret variable (ensure the label matches what you set in Kaggle Secrets)
        # METHOD 1: Directly use the key with wandb.login (Recommended for clarity)
        wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")
        wandb.login(key=wandb_api_key)
        print("Wandb login successful using Kaggle Secret.")

        # METHOD 2: Set environment variable (wandb.init often picks this up automatically)
        # os.environ["WANDB_API_KEY"] = user_secrets.get_secret("WANDB_API_KEY")
        # print("WANDB_API_KEY environment variable set from Kaggle Secret.")
        # If using Method 2, wandb.login() might not even be strictly necessary
        # as wandb.init() usually checks this environment variable.

    except ImportError:
        print("kaggle_secrets not found, assuming API key is set elsewhere (e.g., environment variable).")
    except Exception as e:
        print(f"Error retrieving WANDB_API_KEY from Kaggle Secrets or logging in: {e}")
        # Decide how to handle this - maybe exit, maybe proceed if key might be elsewhere
        # For now, we'll proceed, but wandb logging might fail.
        pass # Or raise e if login is critical


    # --- Initialize WandB Run ---
    run_name = f"{config['model_type']}-e{config['epochs']}-bs{config['batch_size']}-lr{config['learning_rate']}"
    # wandb.init should now work without prompting if login was successful or env var is set
    wandb.init(
        project="sentiment-analysis-hybrid",  # Replace with your project name
        config=config,
        name=run_name,
        job_type="train",
    )
    config = wandb.config # Copy back config
    # === WANDB INTEGRATION END ===
    
    # 1. Get the base model instance (moved to primary device by get_model)
    model = get_model(config)
    primary_device = config['device']
    num_gpus = torch.cuda.device_count()

    # === WANDB INTEGRATION START ===
    # --- Watch the model (optional but useful for gradients/parameters) ---
    # Apply wandb.watch *before* DataParallel wrapping if possible
    # Note: Watching DataParallel models might require watching model.module later
    try:
        # Log gradients and parameters of the model
        wandb.watch(model, log="all", log_freq=100) # Log every 100 batches
    except Exception as e:
        print(f"Could not watch model with wandb: {e}")
    # === WANDB INTEGRATION END ===

    # 2. (Optional) Visualize Architecture *before* wrapping
    if torchviz and len(train_dataloader) > 0:
        print("Attempting model visualization...")
        # Pass the unwrapped model
        visualize_model(model, train_dataloader, config, filename=f"{config['model_type']}_graph")
       
        # === WANDB INTEGRATION START ===
        # Log the model graph image (if created)
        try:
            graph_filename = f"{config['model_type']}_graph.png"
            if os.path.exists(graph_filename):
                 wandb.log({"model_architecture": wandb.Image(graph_filename)})
        except Exception as e:
            print(f"Wandb: Could not log model architecture image: {e}")
        # === WANDB INTEGRATION END ===

    # 3. Wrap model with DataParallel *before* training if multiple GPUs available
    if num_gpus > 1 and primary_device != 'cpu':
        print(f"Wrapping model with nn.DataParallel for training across {num_gpus} GPUs.")
        model = nn.DataParallel(model)
        # DataParallel will manage distribution to devices cuda:0, cuda:1, ...
        # Model should already be on cuda:0 from get_model

    # Ensure model is on the correct device (important if CPU or single GPU)
    model.to(primary_device)

    # 4. Latih Model (pass the potentially wrapped model)
    # run_training now returns the potentially wrapped model and history
    trained_model, history = run_training(config, model, train_dataloader, val_dataloader)

    # 5. Evaluasi Model Terbaik di Test Set
    # The evaluation function now handles loading the saved *base* state dict
    # and potentially wrapping the model again for evaluation.
    evaluate_model_on_test(config, test_dataloader, label_encoder)

    print("\n--- Analysis Complete ---")

    # === WANDB INTEGRATION START ===
    # --- Finish the WandB run ---
    wandb.finish()
    # === WANDB INTEGRATION END ===


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: maybeitsai (hkacode). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Wandb login successful using Kaggle Secret.


Initializing model: bert


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Attempting model visualization...
Model graph saved to bert_graph.png
Wrapping model with nn.DataParallel for training across 2 GPUs.

--- Starting Training ---
Training with DataParallel on 2 GPUs.

Epoch 1/3


...

Evaluating:   0%|          | 0/157 [00:00<?, ?it/s]

Train Loss: 0.1063, Train Acc: 0.9623
Val Loss: 0.1647, Val Acc: 0.9421, Val ROC AUC: 0.9856
Validation acc did not improve. Patience: 1/3
--- Training Finished ---
Training log saved to training_log.json
Training history plot saved to training_history.png

--- Evaluating on Test Set ---
Initializing model: bert
Loading best model state from: best_model.safetensors
Model state loaded successfully.
Using 2 GPUs for evaluation! Wrapping model with DataParallel.


Evaluating:   0%|          | 0/235 [00:00<?, ?it/s]


Test Loss: 0.1433
Test Accuracy: 0.9465
Test ROC AUC: 0.9867

Classification Report:
              precision    recall  f1-score   support

    negative     0.9462    0.9468    0.9465     15000
    positive     0.9468    0.9461    0.9464     15000

    accuracy                         0.9465     30000
   macro avg     0.9465    0.9465    0.9465     30000
weighted avg     0.9465    0.9465    0.9465     30000


Confusion Matrix:
Confusion matrix saved to confusion_matrix_test.png

--- Analysis Complete ---


epoch,▁▅█
train/accuracy,▁▆█
train/loss,█▄▁
val/accuracy,▁█▃
val/loss,█▁█
val/roc_auc,▁▇█
epoch,3
test/accuracy,0.94647
test/loss,0.14335
test/roc_auc,0.98669
train/accuracy,0.96226


## BERT-LSTM-CNN

In [15]:
if __name__ == "__main__":
    # === WANDB INTEGRATION START ===
    # --- Log in to WandB using Kaggle Secrets ---
    try:
        from kaggle_secrets import UserSecretsClient
        import os # Import os to potentially set environment variable

        user_secrets = UserSecretsClient()

        # Retrieve the secret variable (ensure the label matches what you set in Kaggle Secrets)
        # METHOD 1: Directly use the key with wandb.login (Recommended for clarity)
        wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")
        wandb.login(key=wandb_api_key)
        print("Wandb login successful using Kaggle Secret.")

        # METHOD 2: Set environment variable (wandb.init often picks this up automatically)
        # os.environ["WANDB_API_KEY"] = user_secrets.get_secret("WANDB_API_KEY")
        # print("WANDB_API_KEY environment variable set from Kaggle Secret.")
        # If using Method 2, wandb.login() might not even be strictly necessary
        # as wandb.init() usually checks this environment variable.

    except ImportError:
        print("kaggle_secrets not found, assuming API key is set elsewhere (e.g., environment variable).")
    except Exception as e:
        print(f"Error retrieving WANDB_API_KEY from Kaggle Secrets or logging in: {e}")
        # Decide how to handle this - maybe exit, maybe proceed if key might be elsewhere
        # For now, we'll proceed, but wandb logging might fail.
        pass # Or raise e if login is critical


    # --- Initialize WandB Run ---
    run_name = f"{config['model_type']}-e{config['epochs']}-bs{config['batch_size']}-lr{config['learning_rate']}"
    # wandb.init should now work without prompting if login was successful or env var is set
    wandb.init(
        project="sentiment-analysis-hybrid",  # Replace with your project name
        config=config,
        name=run_name,
        job_type="train",
    )
    config = wandb.config # Copy back config
    # === WANDB INTEGRATION END ===
    
    # 1. Get the base model instance (moved to primary device by get_model)
    model = get_model(config)
    primary_device = config['device']
    num_gpus = torch.cuda.device_count()

    # === WANDB INTEGRATION START ===
    # --- Watch the model (optional but useful for gradients/parameters) ---
    # Apply wandb.watch *before* DataParallel wrapping if possible
    # Note: Watching DataParallel models might require watching model.module later
    try:
        # Log gradients and parameters of the model
        wandb.watch(model, log="all", log_freq=100) # Log every 100 batches
    except Exception as e:
        print(f"Could not watch model with wandb: {e}")
    # === WANDB INTEGRATION END ===

    # 2. (Optional) Visualize Architecture *before* wrapping
    if torchviz and len(train_dataloader) > 0:
        print("Attempting model visualization...")
        # Pass the unwrapped model
        visualize_model(model, train_dataloader, config, filename=f"{config['model_type']}_graph")
       
        # === WANDB INTEGRATION START ===
        # Log the model graph image (if created)
        try:
            graph_filename = f"{config['model_type']}_graph.png"
            if os.path.exists(graph_filename):
                 wandb.log({"model_architecture": wandb.Image(graph_filename)})
        except Exception as e:
            print(f"Wandb: Could not log model architecture image: {e}")
        # === WANDB INTEGRATION END ===

    # 3. Wrap model with DataParallel *before* training if multiple GPUs available
    if num_gpus > 1 and primary_device != 'cpu':
        print(f"Wrapping model with nn.DataParallel for training across {num_gpus} GPUs.")
        model = nn.DataParallel(model)
        # DataParallel will manage distribution to devices cuda:0, cuda:1, ...
        # Model should already be on cuda:0 from get_model

    # Ensure model is on the correct device (important if CPU or single GPU)
    model.to(primary_device)

    # 4. Latih Model (pass the potentially wrapped model)
    # run_training now returns the potentially wrapped model and history
    trained_model, history = run_training(config, model, train_dataloader, val_dataloader)

    # 5. Evaluasi Model Terbaik di Test Set
    # The evaluation function now handles loading the saved *base* state dict
    # and potentially wrapping the model again for evaluation.
    evaluate_model_on_test(config, test_dataloader, label_encoder)

    print("\n--- Analysis Complete ---")

    # === WANDB INTEGRATION START ===
    # --- Finish the WandB run ---
    wandb.finish()
    # === WANDB INTEGRATION END ===


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: riqalter (masriq). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Wandb login successful using Kaggle Secret.


Initializing model: bert_lstm_cnn


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Attempting model visualization...
Model graph saved to bert_lstm_cnn_graph.png
Wrapping model with nn.DataParallel for training across 2 GPUs.

--- Starting Training ---
Training with DataParallel on 2 GPUs.

Epoch 1/3


...

Evaluating:   0%|          | 0/157 [00:00<?, ?it/s]

Train Loss: 0.1119, Train Acc: 0.9614
Val Loss: 0.1551, Val Acc: 0.9429, Val ROC AUC: 0.9859
Validation acc did not improve. Patience: 1/3
--- Training Finished ---
Training log saved to training_log.json
Training history plot saved to training_history.png

--- Evaluating on Test Set ---
Initializing model: bert_lstm_cnn
Loading best model state from: best_model.safetensors
Model state loaded successfully.
Using 2 GPUs for evaluation! Wrapping model with DataParallel.


Evaluating:   0%|          | 0/235 [00:00<?, ?it/s]


Test Loss: 0.1466
Test Accuracy: 0.9449
Test ROC AUC: 0.9866

Classification Report:
              precision    recall  f1-score   support

    negative     0.9310    0.9611    0.9458     15000
    positive     0.9598    0.9287    0.9440     15000

    accuracy                         0.9449     30000
   macro avg     0.9454    0.9449    0.9449     30000
weighted avg     0.9454    0.9449    0.9449     30000


Confusion Matrix:
Confusion matrix saved to confusion_matrix_test.png

--- Analysis Complete ---


epoch,▁▅█
train/accuracy,▁▆█
train/loss,█▃▁
val/accuracy,▁█▁
val/loss,█▁▃
val/roc_auc,▁▆█
epoch,3
test/accuracy,0.94493
test/loss,0.14657
test/roc_auc,0.98662
train/accuracy,0.96136


## BERT-LSTM-GCN

In [32]:
if __name__ == "__main__":
    # === WANDB INTEGRATION START ===
    # --- Log in to WandB using Kaggle Secrets ---
    try:
        from kaggle_secrets import UserSecretsClient
        import os # Import os to potentially set environment variable

        user_secrets = UserSecretsClient()

        # Retrieve the secret variable (ensure the label matches what you set in Kaggle Secrets)
        # METHOD 1: Directly use the key with wandb.login (Recommended for clarity)
        wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")
        wandb.login(key=wandb_api_key)
        print("Wandb login successful using Kaggle Secret.")

        # METHOD 2: Set environment variable (wandb.init often picks this up automatically)
        # os.environ["WANDB_API_KEY"] = user_secrets.get_secret("WANDB_API_KEY")
        # print("WANDB_API_KEY environment variable set from Kaggle Secret.")
        # If using Method 2, wandb.login() might not even be strictly necessary
        # as wandb.init() usually checks this environment variable.

    except ImportError:
        print("kaggle_secrets not found, assuming API key is set elsewhere (e.g., environment variable).")
    except Exception as e:
        print(f"Error retrieving WANDB_API_KEY from Kaggle Secrets or logging in: {e}")
        # Decide how to handle this - maybe exit, maybe proceed if key might be elsewhere
        # For now, we'll proceed, but wandb logging might fail.
        pass # Or raise e if login is critical


    # --- Initialize WandB Run ---
    run_name = f"{config['model_type']}-e{config['epochs']}-bs{config['batch_size']}-lr{config['learning_rate']}"
    # wandb.init should now work without prompting if login was successful or env var is set
    wandb.init(
        project="sentiment-analysis-hybrid",  # Replace with your project name
        config=config,
        name=run_name,
        job_type="train",
    )
    config = wandb.config # Copy back config
    # === WANDB INTEGRATION END ===
    
    # 1. Get the base model instance (moved to primary device by get_model)
    model = get_model(config)
    primary_device = config['device']
    num_gpus = torch.cuda.device_count()

    # === WANDB INTEGRATION START ===
    # --- Watch the model (optional but useful for gradients/parameters) ---
    # Apply wandb.watch *before* DataParallel wrapping if possible
    # Note: Watching DataParallel models might require watching model.module later
    try:
        # Log gradients and parameters of the model
        wandb.watch(model, log="all", log_freq=100) # Log every 100 batches
    except Exception as e:
        print(f"Could not watch model with wandb: {e}")
    # === WANDB INTEGRATION END ===

    # 2. (Optional) Visualize Architecture *before* wrapping
    if torchviz and len(train_dataloader) > 0:
        print("Attempting model visualization...")
        # Pass the unwrapped model
        visualize_model(model, train_dataloader, config, filename=f"{config['model_type']}_graph")
       
        # === WANDB INTEGRATION START ===
        # Log the model graph image (if created)
        try:
            graph_filename = f"{config['model_type']}_graph.png"
            if os.path.exists(graph_filename):
                 wandb.log({"model_architecture": wandb.Image(graph_filename)})
        except Exception as e:
            print(f"Wandb: Could not log model architecture image: {e}")
        # === WANDB INTEGRATION END ===

    # 3. Wrap model with DataParallel *before* training if multiple GPUs available
    if num_gpus > 1 and primary_device != 'cpu':
        print(f"Wrapping model with nn.DataParallel for training across {num_gpus} GPUs.")
        model = nn.DataParallel(model)
        # DataParallel will manage distribution to devices cuda:0, cuda:1, ...
        # Model should already be on cuda:0 from get_model

    # Ensure model is on the correct device (important if CPU or single GPU)
    model.to(primary_device)

    # 4. Latih Model (pass the potentially wrapped model)
    # run_training now returns the potentially wrapped model and history
    trained_model, history = run_training(config, model, train_dataloader, val_dataloader)

    # 5. Evaluasi Model Terbaik di Test Set
    # The evaluation function now handles loading the saved *base* state dict
    # and potentially wrapping the model again for evaluation.
    evaluate_model_on_test(config, test_dataloader, label_encoder)

    print("\n--- Analysis Complete ---")

    # === WANDB INTEGRATION START ===
    # --- Finish the WandB run ---
    wandb.finish()
    # === WANDB INTEGRATION END ===


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Wandb login successful using Kaggle Secret.
Initializing model: bert_lstm_gcn
Attempting model visualization...
Model graph saved to bert_lstm_gcn_graph.png
Wrapping model with nn.DataParallel for training across 2 GPUs.

--- Starting Training ---
Training with DataParallel on 2 GPUs.

Epoch 1/3


Training:   0%|          | 0/782 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/157 [00:00<?, ?it/s]

Train Loss: 0.2776, Train Acc: 0.8842
Val Loss: 0.2526, Val Acc: 0.9338, Val ROC AUC: 0.9816
Validation acc improved (0.9338 -> 0.9338). Saving model...
Best model saved to best_model.safetensors
Wandb: Error logging model artifact: metadata must be dict, not <class 'wandb.sdk.wandb_config.Config'>

Epoch 2/3


...

Evaluating:   0%|          | 0/157 [00:00<?, ?it/s]

Train Loss: 0.1132, Train Acc: 0.9603
Val Loss: 0.1875, Val Acc: 0.9454, Val ROC AUC: 0.9842
Validation acc improved (0.9454 -> 0.9454). Saving model...
Best model saved to best_model.safetensors
Wandb: Error logging model artifact: metadata must be dict, not <class 'wandb.sdk.wandb_config.Config'>
--- Training Finished ---
Training log saved to training_log.json
Training history plot saved to training_history.png

--- Evaluating on Test Set ---
Initializing model: bert_lstm_gcn
Loading best model state from: best_model.safetensors
Model state loaded successfully.
Using 2 GPUs for evaluation! Wrapping model with DataParallel.


Evaluating:   0%|          | 0/235 [00:00<?, ?it/s]


Test Loss: 0.1851
Test Accuracy: 0.9430
Test ROC AUC: 0.9852

Classification Report:
              precision    recall  f1-score   support

    negative     0.9433    0.9426    0.9429     15000
    positive     0.9426    0.9433    0.9430     15000

    accuracy                         0.9430     30000
   macro avg     0.9430    0.9430    0.9430     30000
weighted avg     0.9430    0.9430    0.9430     30000


Confusion Matrix:
Confusion matrix saved to confusion_matrix_test.png

--- Analysis Complete ---


epoch,▁▅█
train/accuracy,▁▆█
train/loss,█▃▁
val/accuracy,▁██
val/loss,█▄▁
val/roc_auc,▁█▆
epoch,3
test/accuracy,0.94297
test/loss,0.18509
test/roc_auc,0.98519
train/accuracy,0.96031
